You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
#!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [2]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [17]:
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="https://huggingface.co/meta-llama/Meta-Llama-3-8B",
    huggingfacehub_api_token="",
    task="text-generation",
)

In [4]:
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [22]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True,
    # llm=llm
)

### Agent: Writer

In [23]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True,
    # llm=llm
)

### Agent: Editor

In [24]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True,
    # llm=llm
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [25]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [26]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 3 or 4 paragraphs.",
    agent=writer,
)

### Task: Edit

In [27]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 3 or 4 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [30]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

2024-05-23 12:01:03,701 - 139964733183040 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [12]:
result = crew.kickoff(inputs={"topic": "Machine Learning in Education"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Machine Learning in Education.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer:

Content Plan: Machine Learning in Education

Introduction:
- Overview of Machine Learning in Education
- Importance and benefits of implementing machine learning in educational settings
- Brief mention of key players and recent trends in the field

Key Points:
1. Latest Trends in Machine Learning in Education:
- Personalized learning experiences
- Automated grading and feedback systems
- Predictive analytics for student performance
- Virtual tutors and chatbots for student support

2. Key Players in the

- Display the results of your execution as markdown in the notebook.

In [13]:
from IPython.display import Markdown
Markdown(result)

# Machine Learning in Education: Revolutionizing the Learning Experience

## Introduction to Machine Learning in Education

Machine Learning, a subset of Artificial Intelligence (AI), has been making significant strides in transforming various industries, and education is no exception. By leveraging data analysis and algorithms, machine learning can personalize learning experiences, automate grading systems, provide predictive analytics for student performance, and even offer virtual tutors and chatbots for student support. With key players like Google Education, Microsoft Education, IBM Watson Education, Carnegie Learning, and Knewton leading the way, the integration of machine learning in educational settings is becoming increasingly prevalent.

## Latest Trends and Innovations

One of the most prominent trends in machine learning in education is the shift towards personalized learning experiences. By tailoring educational content to the individual needs and learning styles of students, machine learning can enhance engagement and improve outcomes. Additionally, automated grading and feedback systems save educators time and provide students with immediate insights into their performance. Predictive analytics for student performance help identify at-risk students early on, allowing for timely interventions. Virtual tutors and chatbots offer round-the-clock support to students, answering questions and providing guidance whenever needed.

## Key Players in the Industry

Google Education, Microsoft Education, IBM Watson Education, Carnegie Learning, and Knewton are at the forefront of integrating machine learning in education. These key players are driving innovation and shaping the future of learning with their cutting-edge technologies and solutions. By partnering with educational institutions and investing in research and development, these companies are paving the way for a more personalized and effective learning experience for students worldwide.

## Noteworthy News and Research

Recent research studies have highlighted the effectiveness of machine learning in improving student outcomes. By implementing machine learning algorithms in online learning platforms, educational institutions are seeing significant improvements in student engagement and achievement. Partnerships between schools and tech companies for machine learning initiatives are also on the rise, further accelerating the adoption of this technology in education.

In conclusion, machine learning is revolutionizing the education sector by offering personalized learning experiences, automated grading systems, predictive analytics for student performance, and virtual support through tutors and chatbots. With key players like Google Education, Microsoft Education, IBM Watson Education, Carnegie Learning, and Knewton leading the way, the future of education is increasingly data-driven and tailored to individual student needs. By staying informed on the latest trends and innovations in machine learning in education, educators and educational technology professionals can effectively integrate these technologies into their teaching practices and improve student outcomes.

## Try it with huggingface

- Pass in a topic of your choice and see what the agents come up with!

In [31]:
result = crew.kickoff(inputs={"topic": "Machine Learning in High Education"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Machine Learning in High Education.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer:
Content Plan: Machine Learning in High Education

Title: The Impact of Machine Learning on Higher Education

Outline:
I. Introduction
- Brief explanation of machine learning
- Importance of machine learning in higher education
- Thesis statement highlighting the benefits and challenges of implementing machine learning in high education

II. Latest Trends in Machine Learning in Higher Education
- Overview of current trends in machine learning technology
- Examples of universities or institutions adop

In [32]:
from IPython.display import Markdown
Markdown(result)

# The Impact of Machine Learning on Higher Education

In the rapidly evolving landscape of education, machine learning has emerged as a powerful tool with the potential to revolutionize the way students learn and educators teach. Machine learning, a subset of artificial intelligence that enables systems to learn and improve from experience without being explicitly programmed, holds great promise for higher education. By analyzing data, identifying patterns, and making predictions, machine learning can enhance personalized learning experiences, streamline administrative processes, and drive academic success. However, the implementation of machine learning in higher education also comes with its own set of benefits and challenges.

Universities and institutions are increasingly incorporating machine learning technology into their educational programs to improve student outcomes and enhance the learning experience. From predictive analytics for student retention to personalized learning platforms, the applications of machine learning in higher education are diverse and impactful. Case studies showcasing successful implementations of machine learning highlight its potential to transform traditional teaching methods and improve student engagement and performance.

Leading companies and organizations specializing in machine learning for education are at the forefront of innovation in the field. Through interviews with experts in the intersection of machine learning and education, insights into the potential future impact of machine learning on higher education can be gained. These key players are driving advancements in educational technology and shaping the future of learning.

Recent developments in machine learning technology for educational purposes are continuously shaping the landscape of higher education. Research studies and reports on the effectiveness of machine learning in higher education provide valuable insights into its impact on student learning outcomes. Ethical considerations and privacy concerns related to machine learning in education are also critical aspects that need to be addressed to ensure responsible and equitable use of technology in learning environments.

Students, educators, and administrators are the primary target audience for exploring the potential applications of machine learning in higher education. Students are interested in leveraging technology to enhance their educational experience, while educators seek to understand how machine learning can improve their teaching methods. Administrators are exploring the possibilities of using machine learning to optimize institutional operations and drive better student outcomes.

Research papers, articles, and academic journals focusing on machine learning in higher education provide valuable insights into the latest trends and developments in the field. Websites of leading companies providing machine learning solutions for education offer practical resources and case studies for educators and administrators. By exploring these resources, readers can deepen their understanding of the potential impact of machine learning on higher education.

I encourage readers to explore further resources on machine learning in higher education to stay informed about the latest advancements in educational technology. Your feedback and comments on the potential applications of machine learning in your own educational experiences are invaluable in shaping the future of learning. Let's continue to explore the intersection of technology and education to drive innovation and excellence in higher education.

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).